In [23]:
!pip install qiskit qiskit_aer


In [18]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np


# Helper: Oracle Function (Unchanged)
def oracle(qc, n, marked_state):
    """Constructs the oracle for the marked state."""
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

# Helper: Diffuser Function (Unchanged)
def diffuser(qc, n):
    """Implements the Grover diffuser."""
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))

# Helper: Runner Function for ideal simulation
def run_grover(qc, shots=1024):
    """Executes the Grover circuit and displays the result."""
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=shots).result()
    counts = result.get_counts()
    print('Counts:', counts)
    plot_histogram(counts)
    plt.show()

# Helper: Function to calculate optimal iterations
def optimal_iterations(n):
    """Calculates the optimal number of Grover iterations for n qubits."""
    N = 2**n
    return int(np.floor(np.pi / 4 * np.sqrt(N)))

In [19]:
#TASK 1

# --- Example: Searching for '001' (3 Qubits) ---
marked_state_3q = "001"
n_3q = len(marked_state_3q)
R_3q = optimal_iterations(n_3q) # Should be 2 for N=8 states

print(f"\n--- 3 Qubit Grover Circuit for '{marked_state_3q}' (R={R_3q}) ---")
qc_3q = grover_search_modified(marked_state_3q, R_3q)
print(qc_3q.draw(fold=-1))

# Run the ideal simulation and observe the histogram peak at '001'
run_grover(qc_3q, shots=4096)


# --- Example: Searching for '111' (3 Qubits) ---
marked_state_3q_b = "111"

print(f"\n--- 3 Qubit Grover Circuit for '{marked_state_3q_b}' (R={R_3q}) ---")
qc_3q_b = grover_search_modified(marked_state_3q_b, R_3q)
# Run the ideal simulation and observe the histogram peak at '111'
run_grover(qc_3q_b, shots=4096)


--- 3 Qubit Grover Circuit for '001' (R=2) ---
     ┌───┐ Iteration 1 ┌───┐     ┌───┐ Iteration 1 Diffuser ┌───┐┌───┐          ┌───┐┌───┐      Iteration 2 ┌───┐     ┌───┐ Iteration 2 Diffuser ┌───┐┌───┐          ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├──────░──────┤ X ├──■──┤ X ├──────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────────░──────┤ X ├──■──┤ X ├──────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤      ░      ├───┤  │  ├───┤          ░           ├───┤├───┤       │  ├───┤├───┤           ░      ├───┤  │  ├───┤          ░           ├───┤├───┤       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├──────░──────┤ X ├──■──┤ X ├──────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────────░──────┤ X ├──■──┤ X ├──────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤      ░      ├───┤┌─┴─┐├───┤          ░           ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐      ░      ├───┤┌─┴─┐├───┤          ░           ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_

In [20]:
#TASK 2 & 3
def grover_search_modified(marked_state, iterations):
    """Builds Grover's search circuit for a given marked state and number of iterations."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)

    # 1. Initialization (Hadamard on all qubits)
    qc.h(range(n))

    # 2. R iterations of Oracle and Diffuser
    for r in range(iterations):
        qc.barrier(label=f"Iteration {r+1}")
        oracle(qc, n, marked_state)
        qc.barrier(label=f"Iteration {r+1} Diffuser")
        diffuser(qc, n)

    # 3. Measurement
    qc.measure(range(n), range(n))
    return qc

# --- Example for 4 Qubits (Task 2) and Optimal Iterations (Task 3) ---
marked_state_4q = "1010"
n_4q = len(marked_state_4q)
R_4q = optimal_iterations(n_4q) # Should be 3 for N=16 states

print(f"--- 4 Qubit Grover Circuit (R={R_4q}) ---")
qc_4q = grover_search_modified(marked_state_4q, R_4q)
print(qc_4q.draw(fold=-1))

# Run the ideal simulation
run_grover(qc_4q, shots=4096)

--- 4 Qubit Grover Circuit (R=3) ---
     ┌───┐ Iteration 1                           Iteration 1 Diffuser ┌───┐┌───┐          ┌───┐┌───┐      Iteration 2                           Iteration 2 Diffuser ┌───┐┌───┐          ┌───┐┌───┐      Iteration 3                           Iteration 3 Diffuser ┌───┐┌───┐          ┌───┐┌───┐     ┌─┐         
q_0: ┤ H ├──────░──────────────────■──────────────────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────────░──────────────────■──────────────────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────────░──────────────────■──────────────────────░───────────┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├─────────
     ├───┤      ░      ┌───┐       │  ┌───┐               ░           ├───┤├───┤       │  ├───┤├───┤           ░      ┌───┐       │  ┌───┐               ░           ├───┤├───┤       │  ├───┤├───┤           ░      ┌───┐       │  ┌───┐               ░           ├───┤├───┤       │  ├───┤├───┤     └╥┘┌─┐      
q_1: ┤ H ├──────░──────┤ X ├───────■──┤

In [29]:
# Visualize the circuit in text form
print(qc.draw())


     ┌───┐          ┌───┐┌───┐               ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤┌───┐  │  ├───┤├───┤┌───┐       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤├───┤┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/════════════════════════════════════════════════════════╩══╩══╩═
                                                             0  1  2 


In [15]:
#TASK 5:

def run_grover_with_noise(qc, shots=4096, p_gate=0.001):
    """Executes the Grover circuit with a simple depolarizing noise model."""

    # Create Noise Model: 0.1% error rate per gate
    noise_model = NoiseModel()
    error_1 = depolarizing_error(p_gate, 1) # 1-qubit gate error
    error_2 = depolarizing_error(p_gate, 2) # 2-qubit gate error

    # Add errors to the H, X, and CNOT gates (CX is part of MCX decomposition)
    noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

    # Initialize simulator with the noise model
    simulator_noise = AerSimulator(noise_model=noise_model)

    compiled_circuit_noise = transpile(qc, simulator_noise)
    result_noise = simulator_noise.run(compiled_circuit_noise, shots=shots).result()
    counts_noise = result_noise.get_counts()

    print(f'\nCounts with Noise (p_gate={p_gate*100:.1f}%):', counts_noise)
    plot_histogram(counts_noise, title=f"Noisy Simulation (p_gate={p_gate*100:.1f}%)")
    plt.show()

    most_frequent = max(counts_noise, key=counts_noise.get)
    print(f"Most frequent measured bitstring: {most_frequent}")


# --- Run the 4-qubit circuit with noise ---
print("\n--- Running 4-Qubit Circuit with Noise ---")
run_grover_with_noise(qc_4q)


--- Running 4-Qubit Circuit with Noise ---

Counts with Noise (p_gate=0.1%): {'1110': 30, '1011': 37, '0010': 23, '1101': 54, '0111': 57, '0011': 46, '0100': 48, '0110': 28, '1000': 38, '1100': 28, '0101': 3538, '1001': 31, '1111': 30, '0001': 40, '0000': 33, '1010': 35}
Most frequent measured bitstring: 0101
